In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

#open pickle files
with open('x_training.pkl', 'rb') as file:
    X_training = pickle.load(file)

with open('y_training.pkl', 'rb') as file:
    y_training = pickle.load(file)

with open('evaluation_x.pkl', 'rb') as file:
    evaluation_x = pickle.load(file)

with open('evaluation_y.pkl', 'rb') as file:
    evaluation_y = pickle.load(file)

with open('Y_pool.pkl', 'rb') as file:
    Y_pool = pickle.load(file)

with open('X_pool', 'rb') as file:
   X_pool = pickle.load(file)

with open('embeddings_unlabeled', 'rb') as file:
    embeddings_unlabeled = pickle.load(file)


In [7]:
import sys
from matplotlib import pyplot as plt
from modAL.models import ActiveLearner
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)  

# initializing the learner
learner = ActiveLearner(
    estimator=SVC(kernel='linear',  probability=True),
    X_training=X_training, y_training=y_training,
    bootstrap_init=True
)

learner.fit(X_training, y_training)

# query for labels
n_queries = 5
accuracy_scores = [learner.score(evaluation_x, evaluation_y)]
for idx in range(n_queries):
    # Query for labels from the pool of unlabeled instances
    query_idx, query_instance = learner.query(X_pool)
    
    # Obtain new labels for the queried instances
    print(embeddings_unlabeled.loc[query_idx]['combined_text'])
    sys.stdout.flush()
    # Get the label from the human annotator
    while True:
        user_input = input("Enter the label (0, 1, 2): ")
        try:
            label = int(user_input)
            if label in [0, 1, 2]:
                break
            else:
                print("Label must be one of the following: 0, 1, 2")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    query_idx = np.array(query_idx, dtype=int)
    
    # Update Y_pool with new labels
    Y_pool[query_idx] = label
    
    # Supply labels for queried instances
    learner.teach(X_pool[query_idx], Y_pool[query_idx], bootstrap=True)

    accuracy_scores.append(learner.score(evaluation_x, evaluation_y))
    print(accuracy_scores)

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 5))
    plt.title('Accuracy of the classifier during the active learning')
    plt.plot(range(n_queries+1), accuracy_scores)
    plt.scatter(range(n_queries+1), accuracy_scores)
    plt.xlabel('number of queries')
    plt.ylabel('accuracy')
    plt.show()

accuracy_df = pd.DataFrame(accuracy_scores)
accuracy_df.to_csv('unbalanced_SVM.csv', index=False)

14981    I Keep getting called lazyPeople keep calling me lazy, but when I press them and ask them why they said that their reply is always *"You're not lazy, but you're easily distracted."* This I agree with, but I want to not be distracted to easily I want to be a hard worker I work hard when I am focused I work so hard that when I am locked in I know others won't be able to out work me. This is only when I am locked in and interested, how do I change this how do I work hard and not become so easily distracted? What have you found works for you and does not work for you?
Name: combined_text, dtype: object
[0.495, 0.475]
4089    Question about getting a little help from my UniversityHey guys, you all were so helpful in my last post. I have two questions. I have a meeting on Thursday with a psychiatrist, hopefully going to get some help. My mom offered to drive all the way to my college to go to the meeting with me so it doesn't appear like I'm just seeking drugs (I'm not, just seems t

KeyError: "None of [Int64Index([1172], dtype='int64')] are in the [index]"